In [ ]:
# INSTALLATION / MISE À JOUR COMFYUI AUTOMATIQUE

import os
import subprocess

COMFY_PATH = "/workspace/ComfyUI"

print("🚀 Vérification de l'installation de ComfyUI...")

if os.path.exists(COMFY_PATH):
    print("📂 ComfyUI détecté. Mise à jour en cours...")
    # On force la mise à jour vers la dernière version
    try:
        # git fetch récupère les infos, git reset --hard force l'état local à correspondre exactement au serveur
        subprocess.run(f"cd {COMFY_PATH} && git fetch origin && git reset --hard origin/master", shell=True, check=True)
        print("✅ Mise à jour réussie (Dernier commit master).")
    except Exception as e:
        print(f"⚠️ Erreur lors de la mise à jour : {e}")
        print("   -> Tentative de git pull classique...")
        subprocess.run(f"cd {COMFY_PATH} && git pull", shell=True)
else:
    print("📥 Première installation (Clonage)...")
    subprocess.run(f"git clone https://github.com/comfyanonymous/ComfyUI.git {COMFY_PATH}", shell=True)
    print("✅ Installation terminée.")

In [ ]:
# === INSTALLATION CUSTOM NODES (Manager + Pack Essentiel) ===

import os
import subprocess
import sys

CUSTOM_NODES_DIR = "/workspace/ComfyUI/custom_nodes"

# Liste des dépôts à installer automatiquement

nodes_to_install = [
    "https://github.com/Comfy-Org/ComfyUI-Manager",             # Le Manager
    "https://github.com/rgthree/rgthree-comfy",                 # Rgthree (Indispensable : Mute, Context)
    "https://github.com/kijai/ComfyUI-KJNodes",                 # KJNodes (Outils puissants)    
    "https://github.com/cubiq/ComfyUI_essentials",              # Essentials (Traitement d'image)    
    "https://github.com/negaga53/comfyui-imgloader",            # ImgLoader (Chargement d'images)
    "https://github.com/pythongosssss/ComfyUI-Custom-Scripts",  # Custom-Scripts (Images Feed)  
    "https://github.com/numz/ComfyUI-SeedVR2_VideoUpscaler",
    "https://github.com/sonnybox/ComfyUI-SuperNodes",
    "https://github.com/ltdrdata/ComfyUI-Impact-Pack",
    "https://github.com/ltdrdata/ComfyUI-Impact-Subpack",
    "https://github.com/ClownsharkBatwing/RES4LYF",
    "https://github.com/giriss/comfy-image-saver",
]

print("🚀 Installation automatisée des Custom Nodes...")

for repo_url in nodes_to_install:
    node_name = repo_url.split("/")[-1]
    node_path = os.path.join(CUSTOM_NODES_DIR, node_name)
    
    # 1. Clonage du dépôt s'il n'existe pas
    if not os.path.exists(node_path):
        print(f"⬇️ Clonage de : {node_name}...")
        try:
            subprocess.run(["git", "clone", repo_url, node_path], check=True, stdout=subprocess.DEVNULL)
            
            # 2. Installation des dépendances (requirements.txt)
            req_path = os.path.join(node_path, "requirements.txt")
            if os.path.exists(req_path):
                print(f"   📦 Installation des dépendances pour {node_name}...")
                subprocess.run([sys.executable, "-m", "pip", "install", "-r", req_path], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
                
        except subprocess.CalledProcessError:
            print(f"   ❌ Erreur lors du clonage de {node_name}")
    else:
        print(f"⏩ [SKIP] {node_name} est déjà installé.")

# Fix spécifique pour VideoHelperSuite qui a parfois besoin de ça
try:
    import imageio
except ImportError:
    subprocess.run([sys.executable, "-m", "pip", "install", "imageio[ffmpeg]"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

print("\n🎉 Terminé ! Tous vos nodes favoris sont prêts.")
print(">>> Relancez la cellule de lancement (ComfyUI) pour les charger.")



In [ ]:
# FLUX FILES DOWNLOAD

import os
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- 1. CONFIGURATION SÉCURISÉE ---
CIVITAI_TOKEN = os.environ.get("CIVITAI_TOKEN", "")
HF_TOKEN = os.environ.get("HF_TOKEN", "")

# Installation silencieuse de aria2
os.system("apt-get update -y > /dev/null && apt-get install -y aria2 > /dev/null")

# --- 2. BASE DE DONNÉES (Flux Core & LoRAs - Chemins Absolus) ---
db_models = {
    "🌊 Flux Core (UNET & Checkpoints)": [
        ("Flux.1 Krea Dev (FP8 Scaled)", "https://civitai.com/api/download/models/2079678?type=Model&format=SafeTensor&size=pruned&fp=fp8", "/workspace/ComfyUI/models/unet", "flux1KreaDevFP8_fp8SCALED.safetensors", "civitai"),
        ("Flux.1 Dev Officiel (FP8)", "https://huggingface.co/Comfy-Org/FLUX.1-dev-ComfyUI-Repackaged/resolve/main/flux1-dev-fp8.safetensors?download=true", "/workspace/ComfyUI/models/unet", "flux1-dev-fp8.safetensors", "hf"),
    ],
    "⚙️ Flux Dépendances (VAE & CLIP)": [
        ("VAE Flux (ae.safetensors)", "https://huggingface.co/black-forest-labs/FLUX.1-schnell/resolve/main/ae.safetensors?download=true", "/workspace/ComfyUI/models/vae", "ae.safetensors", "hf"),
        ("T5 XXL (FP8) - Encodeur Texte", "https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/t5xxl_fp8_e4m3fn.safetensors?download=true", "/workspace/ComfyUI/models/clip", "t5xxl_fp8_e4m3fn.safetensors", "hf"),
        ("CLIP L - Encodeur Texte", "https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/clip_l.safetensors?download=true", "/workspace/ComfyUI/models/clip", "clip_l.safetensors", "hf"),
    ],
    "💄 Flux LoRAs (Body & Styles)": [
        ("FC Flux Perfect Busts", "https://civitai.com/api/download/models/788523?type=Model&format=SafeTensor", "/workspace/ComfyUI/models/loras", "FC Flux Perfect Busts.safetensors", "civitai"),
        ("TKTT Steep Slope Tiddies XXL", "https://civitai.com/api/download/models/746019?type=Model&format=SafeTensor", "/workspace/ComfyUI/models/loras", "TKTT_SteepSlopeTiddiesXXL_FLUX.safetensors", "civitai"),
        ("Venus Body (Thicc)", "https://civitai.com/api/download/models/2244224?type=Model&format=SafeTensor", "/workspace/ComfyUI/models/loras", "venus_body_thicc.safetensors", "civitai"),
        ("Olivia 2025 (Fichier Privé/Local)", "METTRE_URL_ICI_SI_DISPO", "/workspace/ComfyUI/models/loras", "olivia2025.safetensors", "civitai"),
    ]
}

# --- 3. INTERFACE GRAPHIQUE ---
style = {'description_width': 'initial'}
layout = widgets.Layout(width='98%')
checkboxes = {}
output_log = widgets.Output(layout={'border': '1px solid #444', 'height': '200px', 'overflow_y': 'scroll'})

accordion_children = []
titles = []

for category, items in db_models.items():
    vbox_items = []
    select_all = widgets.Button(description="Tout cocher", icon="check-square-o", button_style='info', layout=widgets.Layout(width='150px'))
    def on_select_all(b, cat=category):
        for k, val in checkboxes.items():
            if k.startswith(cat):
                val['cb'].value = True
    select_all.on_click(on_select_all)
    vbox_items.append(select_all)
    
    for item in items:
        name, url, path, filename, src = item
        key = f"{category}|{filename}"
        label = f"{name}"
        cb = widgets.Checkbox(value=False, description=label, style=style, layout=layout)
        checkboxes[key] = {'cb': cb, 'data': item}
        vbox_items.append(cb)
        
    accordion_children.append(widgets.VBox(vbox_items))
    titles.append(category)

accordion = widgets.Accordion(children=accordion_children)
for i, title in enumerate(titles):
    accordion.set_title(i, title)
    accordion.selected_index = 0

btn_download = widgets.Button(
    description='LANCER LE TÉLÉCHARGEMENT (Flux)',
    button_style='success', 
    layout=widgets.Layout(width='100%', height='60px', margin='20px 0px 0px 0px'),
    icon='download'
)

# --- 4. LOGIQUE DE TÉLÉCHARGEMENT ---
def run_download(b):
    output_log.clear_output()
    with output_log:
        print("🚀 Traitement des fichiers Flux...\n")
        count = 0
        for key, val in checkboxes.items():
            if val['cb'].value:
                count += 1
                item = val['data']
                name, url, path, filename, src = item
                
                os.makedirs(path, exist_ok=True)
                full_path = os.path.join(path, filename)
                
                if os.path.exists(full_path) and os.path.getsize(full_path) > 1024:
                    print(f"⏩ [SKIP] {name} est déjà là. (0 data utilisée)")
                    continue
                
                if "METTRE_URL" in url:
                    print(f"⚠️ [SKIP] Pas d'URL fournie pour {name}.")
                    continue

                auth = ""
                final_url = url
                if src == "hf":
                    auth = f'--header="Authorization: Bearer {HF_TOKEN}"'
                elif src == "civitai":
                    sep = "&" if "?" in url else "?"
                    final_url = f"{url}{sep}token={CIVITAI_TOKEN}"
                
                print(f"⬇️ Téléchargement de : {name}")
                cmd = f'aria2c -x 16 -s 16 -k 1M -c {auth} -d "{path}" -o "{filename}" "{final_url}"'
                ret = os.system(cmd)
                if ret == 0: print(f"   ✅ OK.")
                else: print(f"   ⚠️ Erreur (Code {ret}).")
                    
        if count == 0: print("⚠️ Aucune case cochée.")
        else: print("\n🎉 Terminé.")

btn_download.on_click(run_download)

display(widgets.VBox([
    widgets.Label("🌊 INSTALLATION FLUX.1 (Core & LoRAs)"),
    accordion,
    btn_download,
    output_log
]))

In [ ]:
# DISABLE REACTOR NSFW FILTER

import os

# Le chemin exact indiqué dans le guide (reactor_sfw.py)
target_path = "/workspace/ComfyUI/custom_nodes/comfyui-reactor/scripts/reactor_sfw.py"

# Le code patché (Nettoyé des commentaires qui cassaient la syntaxe)
bypass_code = """
from PIL import Image
import logging
import os
# On garde les imports essentiels pour éviter que ReActor ne plante au démarrage
from scripts.reactor_logger import logger

# On déclare que le modèle existe pour empêcher le téléchargement
MODEL_EXISTS = True

def ensure_nsfw_model(nsfwdet_model_path):
    # On ment au système : "Oui, le modèle est là, tout va bien"
    return True

def nsfw_image(img_data, model_path: str):
    # C'est ici que le bypass agit.
    # False = "Ce n'est pas NSFW" (Safe)
    # On renvoie False systématiquement sans même regarder l'image.
    return False
"""

print(f"🔧 Application du patch sur : {target_path}")

# Vérification d'existence du dossier
folder = os.path.dirname(target_path)
if os.path.exists(folder):
    try:
        with open(target_path, "w") as f:
            f.write(bypass_code)
        print("✅ Fichier 'reactor_sfw.py' écrasé avec succès.")
        print("🔓 Le filtre NSFW est maintenant désactivé.")
    except Exception as e:
        print(f"❌ Erreur lors de l'écriture : {e}")
else:
    print(f"❌ Le dossier {folder} n'existe pas. Vérifie l'installation de ReActor.")

# Par sécurité, on patche aussi l'ancien nom de fichier s'il existe (certaines versions varient)
alt_path = "/workspace/ComfyUI/custom_nodes/comfyui-reactor/scripts/reactor_nsfw.py"
if os.path.exists(alt_path):
    with open(alt_path, "w") as f:
        f.write(bypass_code)
    print("✅ Ancien fichier 'reactor_nsfw.py' également patché par sécurité.")

print("\n🚨 IMPORTANT : Tu dois REDÉMARRER ComfyUI (Stop/Start) pour que ce changement prenne effet !")

In [ ]:
# ComfyUI Launch Qwen or Wan 2.2

QWEN 
python main.py --listen 0.0.0.0 --port 8188 --disable-auto-launch --cache-ram 8 --bf16-unet

WAN2.2 
python main.py --listen 0.0.0.0 --port 8188 --disable-auto-launch --fast

In [ ]:
# DÉMARRAGE COMFYUI (TOUT-EN-UN : FIX PYTORCH + DÉPENDANCES)

import os
import subprocess
import sys

# --- CONFIGURATION ---
COMFY_DIR = "/workspace/ComfyUI"
MANAGER_DIR = f"{COMFY_DIR}/custom_nodes/ComfyUI-Manager"

print("🚀 DÉMARRAGE COMFYUI (Mode Réparation & Mise à jour)...")

# 1. INSTALLATION OUTILS SYSTÈME
print("\n🔧 [1/6] Vérification outils système (ffmpeg, aria2)...")
subprocess.run(["apt-get", "update"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
subprocess.run(["apt-get", "install", "-y", "ffmpeg", "aria2"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# 2. RÉPARATION PYTORCH (Le Fix Critique)
# On force la version 2.5.1 pour éviter l'erreur 'is_compiling'
print("\n🔥 [2/6] Mise à jour du Noyau PyTorch (Cela prend ~1 min)...")
# On vire xformers qui cause souvent des conflits avec les nouvelles versions
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "xformers"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
# Installation de la version stable CUDA 12.4
subprocess.check_call([
    sys.executable, "-m", "pip", "install",
    "torch==2.5.1+cu124", "torchvision==0.20.1+cu124", "torchaudio==2.5.1+cu124",
    "--index-url", "https://download.pytorch.org/whl/cu124"
])

# 3. INSTALLATION DÉPENDANCES PYTHON
print("\n📦 [3/6] Installation des librairies manquantes...")
required_packages = [
    "numpy<2",           # Force NumPy 1.x (Crucial)
    "pydantic-settings", # Fix config
    "sentencepiece",     # Fix T5/Wan/Flux
    "aiohttp",           # Fix Manager
    "einops",            # Fix Core
    "av",                # Fix Vidéo
    "torchsde",          # Fix Samplers
    "scipy",             # Fix Math
    "insightface",       # Fix FaceSwap
    "onnxruntime-gpu",   # Fix FaceSwap
    "kornia",            # Fix Image Proc
    "spandrel",          # Fix Upscale
    "color-matcher"      # Fix Wan Color
]
subprocess.check_call([sys.executable, "-m", "pip", "install"] + required_packages)

# Requirements du Manager
if os.path.exists(f"{MANAGER_DIR}/requirements.txt"):
    print("   -> Dépendances ComfyUI-Manager...")
    subprocess.run([sys.executable, "-m", "pip", "install", "-r", f"{MANAGER_DIR}/requirements.txt"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# 4. TÉLÉCHARGEMENT MODÈLES INSIGHTFACE (AntelopeV2)
print("\n📥 [4/6] Vérification des modèles InsightFace...")
antelope_dir = f"{COMFY_DIR}/models/insightface/models/antelopev2"
os.makedirs(antelope_dir, exist_ok=True)
files_antelope = ["1k3d68.onnx", "2d106det.onnx", "genderage.onnx", "glintr100.onnx", "scrfd_10g_bnkps.onnx"]

for f in files_antelope:
    if not os.path.exists(f"{antelope_dir}/{f}"):
        print(f"   ⬇️ Téléchargement : {f}")
        subprocess.run(["aria2c", "-x", "16", "-s", "16", "-k", "1M", "-c", f"https://huggingface.co/okaris/antelopev2/resolve/main/{f}?download=true", "-d", antelope_dir, "-o", f], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# 5. CONFIGURATION FINALE
print("\n⚙️ [5/6] Configuration finale...")
# On s'assure que le dossier custom_nodes est propre si besoin (optionnel)

# 6. LANCEMENT
print("\n🚀 [6/6] Lancement du serveur...")

run_script = """#!/bin/bash
cd /workspace/ComfyUI
# Arguments optimisés pour RunPod
python main.py --listen 0.0.0.0 --port 8188 --disable-auto-launch --fast
"""

with open("run_comfy.sh", "w") as f:
    f.write(run_script)
os.chmod("run_comfy.sh", 0o755)

print("✅ ComfyUI démarre. (Si le terminal bloque ici, c'est bon signe : le serveur tourne !)")
!./run_comfy.sh